In [1]:
import sys
sys.path.append('../../')
import os
import time
import numpy 
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test_recognition_extract as input_test
import math
import numpy as np
from i3d import InceptionI3d
from utils import *
import pandas as pd
import cv2
from PIL import Image
import hickle
import scipy.io as sio

flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1, 'Batch size.')
flags.DEFINE_integer('num_frame_per_clib', 64, 'Nummber of frames per clib')
flags.DEFINE_integer('crop_size', 224, 'Crop_size')
flags.DEFINE_integer('rgb_channels', 2, 'Channels for input')
flags.DEFINE_integer('classics', 14, 'The num of class')
FLAGS = flags.FLAGS

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

outfile2 = '/media/ee401_2/Didik/features/frames_flow_untrim_mat/'
pre_model_save_dir = "/media/ee401_2/Didik/fixed_crime_models/rec_trim_flow_15000_6_0.0001_1000"
test_list_file = '/media/ee401_2/Didik/crime_detection/I3D-Tensorflow/list/ucf_crime_fix/recognition_extract2.list'

if not os.path.exists(outfile2):
    os.makedirs(outfile2)
    
def run_training():
    # Get the sets of images and labels for training, validation, and
    # Tell TensorFlow that the model will be built into the default Graph.
    
    if not os.path.exists(pre_model_save_dir):
        print('WARNING !!!!! check the path model')
        exit()
    else:
        file = list(open(test_list_file, 'r'))
        num_test_videos = len(file)
        print("Number of test videos={}".format(num_test_videos))
        with tf.Graph().as_default():
            _, rgb_images_placeholder, labels_placeholder, is_training = placeholder_inputs(
                            FLAGS.batch_size * gpu_num,
                            FLAGS.num_frame_per_clib,
                            FLAGS.crop_size,
                            FLAGS.rgb_channels
                            )

            with tf.variable_scope('Flow'):
                logit, _ = InceptionI3d(
                                    num_classes=FLAGS.classics,
                                    spatial_squeeze=True,
                                    final_endpoint='Mixed_5c',
                                    name='inception_i3d'
                                    )(rgb_images_placeholder, is_training)
            norm_score = tf.nn.softmax(logit)

            # Create a saver for writing training checkpoints.
            saver = tf.train.Saver()
            init = tf.global_variables_initializer()

            # Create a session for running Ops on the Graph.
            sess = tf.Session(
                            config=tf.ConfigProto(allow_soft_placement=True)
                            )
            sess.run(init)

        ckpt = tf.train.get_checkpoint_state(pre_model_save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            print ("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            print ("load complete!")

        all_steps = num_test_videos
        top1_list = []
        count = 0
        for step in xrange(all_steps):
            start_time = time.time()
            s_index = 0
            predicts = []
            data = {}
            top1 = False
            data_new = np.array([])
            while True:
                _, val_images,  val_labels, s_index, is_end = input_test.read_clip_and_label(
                                filename=file[step],
                                batch_size=FLAGS.batch_size * gpu_num,
                                s_index=s_index,
                                num_frames_per_clip=FLAGS.num_frame_per_clib,
                                crop_size=FLAGS.crop_size,
                                )

                data_features = sess.run(logit,
                                   feed_dict={
                                                rgb_images_placeholder: val_images,
                                                labels_placeholder: val_labels,
                                                is_training: False
                                                })

                
                tmp = np.squeeze(data_features)
                tmp = np.mean(tmp, axis=2)
                tmp = np.mean(tmp, axis=1)
                print(tmp.shape)
                #tmp = np.mean(tmp, axis=1)
                
                
                if data_new.shape[0] == 0:
                    data_new = tmp
                else:
                    data_new = np.vstack((data_new, tmp))

                if is_end:        
                    data['feat'] = data_new
                    temp = file[step].split("/")
                    
                    namefile = temp[7].split(" ")

                    datasave2 = outfile2+namefile[0]
                    sio.savemat(datasave2+'.mat',data, do_compression=False)
                    break
            count = count+1
            
            print('----------------')
            print(count,' >>' ,datasave2)
            print(data_new.shape)
            print('----------------')


def main(_):
    run_training()


if __name__ == '__main__':
    tf.app.run()


/usr/local/lib/python3.5/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Number of test videos=700
net_shape:
(1, 8, 7, 7, 1024)
loading checkpoint /media/ee401_2/Didik/fixed_crime_models/rec_trim_flow_15000_6_0.0001_1000/i3d_hmdb_model-14999,waiting......
INFO:tensorflow:Restoring parameters from /media/ee401_2/Didik/fixed_crime_models/rec_trim_flow_15000_6_0.0001_1000/i3d_hmdb_model-14999
load complete!
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse001_x264...strat_frame = 0
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse001_x264...strat_frame = 64
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse001_x264...strat_frame = 128
(8, 1024)
----------------
1  >> /media/ee401_2/Didik/features/frames_flow_mat/Abuse001_x264
(24, 1024)
----------------
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse002_x264...strat_frame = 0
(8, 1024)
Lo

(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse004_x264...strat_frame = 256
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse004_x264...strat_frame = 320
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse004_x264...strat_frame = 384
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse004_x264...strat_frame = 448
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse004_x264...strat_frame = 512
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse004_x264...strat_frame = 576
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse004_x264...strat_frame = 640
(8, 1024)
Loading a video c

(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse010_x264...strat_frame = 128
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse010_x264...strat_frame = 192
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse010_x264...strat_frame = 256
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse010_x264...strat_frame = 320
(8, 1024)
----------------
10  >> /media/ee401_2/Didik/features/frames_flow_mat/Abuse010_x264
(48, 1024)
----------------
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse011_x264...strat_frame = 0
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Abuse/Abuse011_x264...strat_frame = 64
(8, 1024)
Loading a video clip from /media/ee401_2/Datase

(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Arrest/Arrest001_x264...strat_frame = 320
(8, 1024)
----------------
13  >> /media/ee401_2/Didik/features/frames_flow_mat/Arrest001_x264
(48, 1024)
----------------
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Arrest/Arrest002_x264...strat_frame = 0
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Arrest/Arrest002_x264...strat_frame = 64
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Arrest/Arrest002_x264...strat_frame = 128
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Arrest/Arrest002_x264...strat_frame = 192
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Arrest/Arrest002_x264...strat_frame = 256
(8, 1024)
Loading a video clip from /media/e

...

(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism043_x264...strat_frame = 576
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism043_x264...strat_frame = 640
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism043_x264...strat_frame = 704
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism043_x264...strat_frame = 768
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism043_x264...strat_frame = 832
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism043_x264...strat_frame = 896
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism

(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism045_x264...strat_frame = 1344
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism045_x264...strat_frame = 1408
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism045_x264...strat_frame = 1472
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism045_x264...strat_frame = 1536
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism045_x264...strat_frame = 1600
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism045_x264...strat_frame = 1664
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Van

(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism048_x264...strat_frame = 1216
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism048_x264...strat_frame = 1280
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism048_x264...strat_frame = 1344
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism048_x264...strat_frame = 1408
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism048_x264...strat_frame = 1472
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Vandalism048_x264...strat_frame = 1536
(8, 1024)
Loading a video clip from /media/ee401_2/Dataset2/UCF_crimes/trimmed_ucf_crimes_det_flow/Vandalism/Van

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
